In [1]:
%matplotlib inline

import ctypes

import numpy as np

import radarkit

In [2]:
def place_RKComplex_array_no_negate(dst, src):
    ctypes.memmove(
        ctypes.cast(dst, ctypes.POINTER(ctypes.c_float)),
        src.flatten().ctypes.data,
        src.nbytes,
    )

def place_RKComplex_array_out_place(dst, src):
    src.imag = -src.imag
    ctypes.memmove(
        ctypes.cast(dst, ctypes.POINTER(ctypes.c_float)),
        src.flatten().ctypes.data,
        src.nbytes,
    )

def place_RKComplex_array_in_place(dst, src):
    np.conj(src, out=src)
    ctypes.memmove(
        ctypes.cast(dst, ctypes.POINTER(ctypes.c_float)),
        src.flatten().ctypes.data,
        src.nbytes,
    )

In [3]:
pulses = radarkit.RKBuffer()
pulseBufferSize = radarkit.RKPulseBufferAlloc(ctypes.byref(pulses), 4096, 10)

pulse = radarkit.RKGetPulseFromBuffer(pulses, 1)

x = np.random.randn(2300).astype(np.complex64)

In [4]:
%timeit place_RKComplex_array_no_negate(radarkit.RKGetComplexDataFromPulse(pulse, 0), x)
%timeit place_RKComplex_array_out_place(radarkit.RKGetComplexDataFromPulse(pulse, 0), x)
%timeit place_RKComplex_array_in_place(radarkit.RKGetComplexDataFromPulse(pulse, 0), x)

3.02 µs ± 19.4 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
4.93 µs ± 35.8 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
3.63 µs ± 45.7 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [5]:
def read_RKComplex_from_pulse(pulse, count):
    h = ctypes.cast(radarkit.RKGetComplexDataFromPulse(pulse, 0), ctypes.POINTER(ctypes.c_float))
    h = np.ctypeslib.as_array(h, (count * 2,)).view(np.complex64)
    v = ctypes.cast(radarkit.RKGetComplexDataFromPulse(pulse, 1), ctypes.POINTER(ctypes.c_float))
    v = np.ctypeslib.as_array(v, (count * 2,)).view(np.complex64)
    x = np.vstack((h, v))
    np.conj(x, out=x)
    return x


def read_RKComplex_from_pulse_v2(pulse, count):
    _from_mem = ctypes.pythonapi.PyMemoryView_FromMemory
    _from_mem.restype = ctypes.py_object
    h = np.frombuffer(_from_mem(radarkit.RKGetComplexDataFromPulse(pulse, 0), 8*count), dtype=np.complex64)
    v = np.frombuffer(_from_mem(radarkit.RKGetComplexDataFromPulse(pulse, 1), 8*count), dtype=np.complex64)
    x = np.vstack((h, v))
    np.conj(x, out=x)
    return x

In [6]:
place_RKComplex_array_in_place(radarkit.RKGetComplexDataFromPulse(pulse, 0), x)

In [7]:
%timeit y = read_RKComplex_from_pulse(pulse, 2300)
%timeit y = read_RKComplex_from_pulse_v2(pulse, 2300)

8.36 µs ± 59 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
5.29 µs ± 359 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
